## A project to identify suitable location(s) for constructing warehouse (Recommender System)

In [2]:
# importing required libraries 
import numpy as np  
import pandas as pd  
from bs4 import BeautifulSoup
import requests  
import json  
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if the library is already installed
import geopy.geocoders # convert an address into latitude and longitude values

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if the library is already installed
import folium # map rendering library

print('Required Libraries are imported.')

Required Libraries are imported.


### Toronto postal codes details. The same is available in a CSV file.

In [3]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head()

,Unnamed: 0,Postal code,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


###  Map of Toronto city with its postal codes as regions.

In [5]:
#toronto_latitude = 43.6932; toronto_longitude = -79.3832
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude

Toronto_map = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'],df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

### We select  "Scarorough" borough from Toronto city and its neighborhoods

In [6]:
# Filering only neighborhoods belongs to "Scarborough" borough.
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Making a map of Scarborough and Its neighbourhoods

In [7]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
#print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough



## Function to access 'Foursquare' data (explore api)

In [8]:
def foursquare_search (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Received, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;


### Storing 'Foursquare' credentials to variables.

In [30]:
CLIENT_ID = 'QHUMOXGWQVJDC2CHA35N4R03TDDW2DT5DZXZ1D40BZQR4JG1' #  Foursquare ID
CLIENT_SECRET = 'NYOPDDLYL40X4XFRNDLBS0TKRYRPEKLRECZUTA2CTALWWV5Q' # Foursquar Secret
VERSION = '20180605' # Foursquare API version

### Accessing 'Foursquare' and obtaining venues in the neighborhoods inside 'Scarborough'

In [31]:
print('Start accessing different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_search(list(scarborough_data['Postal code']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)
print('')
print('Finished accessing different neighborhoods inside "Scarborough"')

# To avoid connecting to 'Foursquare' everytime the recived data saved to a local file. This can help us to avoid accessing 
# iternet repeatedly.
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
    
print('')
print("Received Data from 'Foursquare' is Saved to a file.")  

with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)


Start accessing different neighborhoods inside "Scarborough"
1.
Data is Received, for the Postal Code M1B (and Neighborhoods Malvern / Rouge) SUCCESSFULLY.
2.
Data is Received, for the Postal Code M1C (and Neighborhoods Rouge Hill / Port Union / Highland Creek) SUCCESSFULLY.
3.
Data is Received, for the Postal Code M1E (and Neighborhoods Guildwood / Morningside / West Hill) SUCCESSFULLY.
4.
Data is Received, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Received, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Received, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Received, for the Postal Code M1K (and Neighborhoods Kennedy Park / Ionview / East Birchmount Park) SUCCESSFULLY.
8.
Data is Received, for the Postal Code M1L (and Neighborhoods Golden Mile / Clairlea / Oakridge) SUCCESSFULLY.
9.
Data is Received, for the Postal Code M1M (and Neighborhoods Cliffside / Cliffcrest / Scarboro

### Cleaning the data received from 'Foursquare' is performed here 

In [32]:
# This function is created to clean every neighborhood inside the dataset.

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        #print('Number of Venues "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        #print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            
    
    return(result_df)

In [33]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)
print ("Cleaning of 'Sacrborough' data received from 'Foursquare' finished here")

Cleaning of 'Sacrborough' data received from 'Foursquare' finished here


### Showing venues for each neighborhood in Scarborough

In [34]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,Malvern / Rouge,43.806686,-79.194353,Harvey's,This spot is popular,Restaurant,807
1,M1B,Malvern / Rouge,43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
2,M1B,Malvern / Rouge,43.806686,-79.194353,Wendy’s,This spot is popular,Fast Food Restaurant,387
3,M1B,Malvern / Rouge,43.806686,-79.194353,RBC Royal Bank,This spot is popular,Bank,906
4,M1B,Malvern / Rouge,43.806686,-79.194353,Caribbean Wave,This spot is popular,Caribbean Restaurant,912


In [35]:
scarborough_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
389,M1W,Steeles West / L'Amoreaux West,43.799525,-79.318389,Pharmacy Ave. & Finch Ave. E,This spot is popular,Intersection,830
390,M1W,Steeles West / L'Amoreaux West,43.799525,-79.318389,L'Amoreaux Scarborough Winter Tennis Club,This spot is popular,Tennis Court,896
391,M1W,Steeles West / L'Amoreaux West,43.799525,-79.318389,Divine Wok Restaurant,This spot is popular,Chinese Restaurant,957
392,M1W,Steeles West / L'Amoreaux West,43.799525,-79.318389,Olympian Swimming,This spot is popular,Gym Pool,978
393,M1W,Steeles West / L'Amoreaux West,43.799525,-79.318389,Dumpling & Szechuan Cuisine（川流不息店）,This spot is popular,Chinese Restaurant,989


## Saving a cleaned final version of  dataframe to a csv file (data from Foursquare) 
## This will be stored in a local file and can be used as and when required.
## The same is again read and loaded

In [36]:
scarborough_venues.to_csv('scarborough_venues.csv')
scarborough_venues = pd.read_csv('scarborough_venues.csv')

### Neighborhood Summary information

In [37]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,44,44,44,44,44,44,44
Birch Cliff / Cliffside West,14,14,14,14,14,14,14
Cedarbrae,29,29,29,29,29,29,29
Clarks Corners / Tam O'Shanter / Sullivan,35,35,35,35,35,35,35
Cliffside / Cliffcrest / Scarborough Village West,12,12,12,12,12,12,12


### One-hot encoding the 'categroies' column into every unique categorical feature.

In [38]:
# one hot encoding
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,African Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Badminton Court,Bakery,Bank,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hardware Store,Hobby Shop,Hockey Arena,Hookah Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,0,M1B,Malvern / Rouge,43.806686,-79.194353,Harvey's,This spot is popular,807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1B,Malvern / Rouge,43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,Malvern / Rouge,43.806686,-79.194353,Wendy’s,This spot is popular,387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,Malvern / Rouge,43.806686,-79.194353,RBC Royal Bank,This spot is popular,906,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1B,Malvern / Rouge,43.806686,-79.194353,Caribbean Wave,This spot is popular,912,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Features relevent to the contractor is selected manually and stored in a list.

In [19]:
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'American Restaurant',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Breakfast Spot',
 'Burger Joint',
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Diner',
 'Fast Food Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Greek Restaurant',
 'Grocery Store',
 'Hakka Restaurant',
 'Hotpot Restaurant',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',
 'Malay Restaurant',
 'Mediterranean Restaurant',
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 'Noodle House',
 'Pizza Place',
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 'Thai Restaurant',
 'Vegetarian / Vegan Restaurant',
 'Vietnamese Restaurant',
 'Wings Joint']

### Updating the One-hot encoded dataframe and  grouping the data by neighborhoods

In [39]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()

scarborough_onehot.head()

,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,0,0,2,1,0,0,1,2,7,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,0,1,1,1,1,1,1,0,0,0,1,0
Birch Cliff / Cliffside West,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
Cedarbrae,0,0,0,3,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
Clarks Corners / Tam O'Shanter / Sullivan,0,0,0,0,0,0,0,1,1,1,0,2,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,1,2,0,2,1,0,1,1,0,1,0
Cliffside / Cliffcrest / Scarborough Village West,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0


## Assuming all resaturants use the same raw materials, integrating all restaurants and joints.

In [40]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []

for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)

feat_name_list = list(scarborough_onehot.columns)
joint_list = []

for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

###  Final dataframe about neighborhoods inside Scarborrough is ready and the same can be used with machine learning algorithms.

In [41]:
# final dataframe
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,
Agincourt,2,1,0,0,0,0,1,1,1,19,0
Birch Cliff / Cliffside West,0,0,1,0,0,0,0,0,0,2,0
Cedarbrae,3,0,0,0,0,1,0,1,0,7,3
Clarks Corners / Tam O'Shanter / Sullivan,0,0,0,0,0,1,1,2,2,12,1
Cliffside / Cliffcrest / Scarborough Village West,0,0,0,0,0,0,0,3,0,1,1
Dorset Park / Wexford Heights / Scarborough Town Centre,2,0,0,0,0,0,0,1,1,14,2
Golden Mile / Clairlea / Oakridge,2,0,2,0,0,1,0,1,1,3,0
Guildwood / Morningside / West Hill,0,0,0,0,1,1,0,3,1,5,1
Kennedy Park / Ionview / East Birchmount Park,0,0,0,0,0,2,0,1,1,7,1


# Using k-means to cluster neighborhoods into 5 clusters

In [42]:
# import k-means
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)


## Displaying centers of each cluster

In [43]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G3,2.000000,1.000000,0.0,0.0,0.00,0.000000,1.00,1.000000,1.000000,19.000000,0.000000,25.000000
G4,1.000000,0.000000,0.0,0.0,0.00,0.666667,1.00,1.666667,1.000000,12.666667,1.333333,19.333333
G2,2.000000,0.500000,0.0,0.5,0.00,2.000000,0.00,1.500000,0.000000,8.000000,2.500000,17.000000
G5,0.750000,0.250000,0.0,0.0,0.25,1.250000,0.25,1.500000,1.000000,6.250000,0.500000,12.000000
G1,0.333333,0.166667,0.5,0.0,0.00,0.333333,0.00,0.833333,0.333333,2.166667,0.333333,5.000000


### From the above it is clear that the best group is G5 followed by G1, G4

### Inserting "kmeans.labels_" into the  Scarborough dataframe and finding corresponding group for each neighborhood.

In [44]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,3
1,Birch Cliff / Cliffside West,1
2,Cedarbrae,2
3,Clarks Corners / Tam O'Shanter / Sullivan,4
4,Cliffside / Cliffcrest / Scarborough Village West,1
5,Dorset Park / Wexford Heights / Scarborough To...,4
6,Golden Mile / Clairlea / Oakridge,1
7,Guildwood / Morningside / West Hill,5
8,Kennedy Park / Ionview / East Birchmount Park,5
9,Malvern / Rouge,5


## First best neighborhood is

In [45]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
7,Guildwood / Morningside / West Hill,5
8,Kennedy Park / Ionview / East Birchmount Park,5
9,Malvern / Rouge,5
13,Steeles West / L'Amoreaux West,5


In [46]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1E',
 'Neighborhood': 'Guildwood / Morningside / West Hill',
 'Neighborhood Latitude': 43.7635726,
 'Neighborhood Longitude': -79.1887115}

## Second best neighborhood is

In [47]:
neigh_summary[neigh_summary['Group'] == 1]

,Neighborhood,Group
1,Birch Cliff / Cliffside West,1
4,Cliffside / Cliffcrest / Scarborough Village West,1
6,Golden Mile / Clairlea / Oakridge,1
11,Rouge Hill / Port Union / Highland Creek,1
12,Scarborough Village,1
15,Woburn,1


In [48]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 1]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1N',
 'Neighborhood': 'Birch Cliff / Cliffside West',
 'Neighborhood Latitude': 43.692657000000004,
 'Neighborhood Longitude': -79.2648481}

## Third best neighborhood is

In [27]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
3,Clarks Corners / Tam O'Shanter / Sullivan,4
5,Dorset Park / Wexford Heights / Scarborough To...,4
10,Milliken / Agincourt North / Steeles East / L'...,4


In [29]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1T',
 'Neighborhood': "Clarks Corners / Tam O'Shanter / Sullivan",
 'Neighborhood Latitude': 43.7816375,
 'Neighborhood Longitude': -79.3043021}